In [93]:
import os
import requests
import re
from fake_useragent import UserAgent

from engines import utilities

import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [104]:
def p2f(x):
    return float(x.strip('%'))/100

def s2f(x):
    return float(x.replace(',',''))

In [95]:
ua = UserAgent()
headers = {"User-Agent": ua.random}
response = requests.get("https://www.fragrantica.com/perfume/Creed/Aventus-9828.html", headers=headers)
print(response.status_code)

200


In [113]:
# URL of a perfume page
url = "https://www.fragrantica.com/perfume/Creed/Aventus-9828.html"
# url = 'https://www.fragrantica.com/perfume/Le-Labo/Tonka-25-51697.html'

In [114]:
def create_perfume_dict(url):
    headers = {"User-Agent": ua.random} # Create a new header every time we do a pull from the API
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
    
        perfume_dict={}
        perfume_dict['url']=url
        
        # Extract data
        perfume_dict['name'] = soup.find("h1").text.strip()
        name = soup.find("h1").text.strip()
        gender=''
        if 'women' in name and 'men' in name:
            gender='unisex'
        elif 'men' in name:
            gender='male'
        elif 'female' in name:
            gender='female'
        
        perfume_dict['gender']=gender
        perfume_dict['brand'] = soup.find("span", {"itemprop": "name"}).text.strip()
        perfume_dict['rating'] = s2f(soup.find("span", {"itemprop": "ratingValue"}).text.strip())
        perfume_dict['vote'] = s2f(soup.find("span", {"itemprop": "ratingCount"}).text.strip())
        
        # Accords
        accords = soup.find_all("div", class_="accord-bar")
        accord_dict = {}
        for accord in accords:
            style=accord['style']
            width_match = re.search(r"width\s*:\s*([^;]+)", style, re.IGNORECASE)
            if width_match:
                width = p2f(width_match.group(1).strip())
            else:
                width = 0
            accord_dict[accord.get_text()]=width
    
        perfume_dict['accords']=accord_dict
        '''
            Get notes saved into a_tags
        '''
        
        a_tags = soup.find_all("a", href=lambda href: href and href.startswith("https://www.fragrantica.com/notes/"))
        note_dict={}
        if a_tags:
            for a_tag in a_tags:
                target_div = a_tag.parent
                note_text = target_div.get_text(strip=True, separator=" ").replace(a_tag.get_text(strip=True), "").strip().lower()
                note_dict[note_text]=a_tag['href']
        else:
            print("No matching <a> tags found.")
        perfume_dict['notes']=note_dict
        return perfume_dict
        
    else:
        print(f"Failed to retrieve page: {response.status_code}")
        return None

In [115]:
perfume_dict = create_perfume_dict(url)

In [116]:
perfume_dict

{'url': 'https://www.fragrantica.com/perfume/Creed/Aventus-9828.html',
 'name': 'Aventus Creed for men',
 'gender': 'male',
 'brand': 'Creed',
 'rating': 4.33,
 'vote': 22410.0,
 'accords': {'fruity': 1.0,
  'sweet': 0.6835487005975099,
  'woody': 0.6501511770210899,
  'leather': 0.63863232788616,
  'smoky': 0.57090502243659,
  'citrus': 0.55675737287932,
  'musky': 0.5384817507738799,
  'tropical': 0.52288532143114,
  'fresh': 0.51834581143666,
  'mossy': 0.46150325150577},
 'notes': {'bergamot': 'https://www.fragrantica.com/notes/Bergamot-75.html',
  'black currant': 'https://www.fragrantica.com/notes/Black-Currant-132.html',
  'apple': 'https://www.fragrantica.com/notes/Apple-146.html',
  'lemon': 'https://www.fragrantica.com/notes/Lemon-77.html',
  'pink pepper': 'https://www.fragrantica.com/notes/Pink-Pepper-91.html',
  'pineapple': 'https://www.fragrantica.com/notes/Pineapple-170.html',
  'patchouli': 'https://www.fragrantica.com/notes/Patchouli-34.html',
  'moroccan jasmine': 'h